## 二、KNN的自定义函数实现
   - 算法实现： （小数据量，线性扫描）
    - https://www.cnblogs.com/hemiy/p/6155425.html
         1. 输入x与训练集各点的距离distance
         2. 按distance排序，取distance最近的k个点（k为用户输入）
         3. 对k个点的类归类计数，x归为多数类（多数表决）
         4. or 对k个点按1/square(distance)权重归类计数，x归为计数大的类（加权表决）

   - 对于大数据量，线性扫描效率极低，于是采用kd树储存训练集，通过搜索kd树的方法寻找输入的近邻，将输入归类（算法如何实现？自定义函数2）

In [ ]:
import pandas as pd
import numpy as np
import operator

### 自定义KNN分类器
 - newInput: 新输入的待分类数据(x_test)，**本分类器一次只能对一个新输入分类**
 - dataset：输入的训练数据集(x_train),array类型，**每一行为一个输入训练集**
 - labels：输入训练集对应的类别标签(y_train)，**格式为['A','B']而不是[['A'],['B']]**
 - k：近邻数
 - weight：决策规则，"uniform" 多数表决法，"distance" 距离加权表决法

In [ ]:
# newInput: 新输入的待分类数据(x_test)，本分类器一次只能对一个新输入分类
# dataset：输入的训练数据集(x_train),array类型，每一行为一个输入训练集
# labels：输入训练集对应的类别标签(y_train)，格式为['A','B']而不是[['A'],['B']]
# k：近邻数
# weight：决策规则，"uniform" 多数表决法，"distance" 距离加权表决法

def KNNClassify(newInput, dataset, labels, k, weight):
    numSamples=dataset.shape[0]
    
    """step1: 计算待分类数据与训练集各数据点的距离（欧氏距离：距离差值平方和开根号）"""
    diff=np.tile(newInput,(numSamples,1)) - dataset # 凸显numpy数组的高效性——元素级的运算
    squaredist=diff**2
    distance = (squaredist.sum(axis=1))**0.5 # axis=1,按行累加
    
    """step2：将距离按升序排序，并取距离最近的k个近邻点"""
    # 对数组distance按升序排序，返回数组排序后的值对应的索引值
    sortedDistance=distance.argsort() 
    
    # 定义一个空字典，存放k个近邻点的分类计数
    classCount={}
    
    # 对k个近邻点分类计数，多数表决法
    for i in range(k):
        # 第i个近邻点在distance数组中的索引,对应的分类
        votelabel=labels[sortedDistance[i]]
        if weight=="uniform":
            # votelabel作为字典的key，对相同的key值累加（多数表决法）
            classCount[votelabel]=classCount.get(votelabel,0)+1 
        elif weight=="distance":
            # 对相同的key值按距离加权累加（加权表决法）
            classCount[votelabel]=classCount.get(votelabel,0)+(1/distance[sortedDistance[i]])
        else:
            print ("分类决策规则错误！")
            print ("\"uniform\"多数表决法\"distance\"距离加权表决法")
            break 
            
    # 对k个近邻点的分类计数按降序排序，返回得票数最多的分类结果
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    if weight=="uniform":
        print ("新输入到训练集的最近%d个点的计数为："%k,"\n",classCount)
        print ("新输入的类别是:", sortedClassCount[0][0])
    
    elif weight=="distance":
        print ("新输入到训练集的最近%d个点的距离加权计数为："%k,"\n",classCount)
        print ("新输入的类别是:", sortedClassCount[0][0])
    
    return sortedClassCount[0][0]

#### 鸢尾花数据集分类测试

In [ ]:
iris=pd.read_csv("E:\python\practice\iris.txt")
iris.head()

,sepallength,sepalwidth,petallength,petalwidth,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### 建立训练集、测试集
  - 注意训练集x、y的格式

In [ ]:
iris_x=iris.iloc[:,[0,1,2,3]]
iris_y=iris.iloc[:,[4]]

np.random.seed(7)
indices=np.random.permutation(len(iris_x))

iris_x_train=iris_x.iloc[indices[0:130]]
iris_y_train=iris_y.iloc[indices[0:130]]

iris_x_test=iris_x.iloc[indices[130:150]]
iris_y_test=iris_y.iloc[indices[130:150]]

# 将dataframe格式的数据转换为numpy array格式，便于调用函数计算
iris_x_train=np.array(iris_x_train)
iris_y_train=np.array(iris_y_train)

iris_x_test=np.array(iris_x_test)
iris_y_test=np.array(iris_y_test) 

print (iris_x_test[1])
print (iris_y_test[1])

"""运行错误测试：
dis=(((np.tile(iris_x_test[1],(130,1))-iris_x_train)**2).sum(axis=1))**0.5
sortdis=dis.argsort()
cc={}
for i in range(10):
    votel=iris_y_train[sortdis[i]]
    cc[votel]=cc.get(votel,0)+1

sortedcc=sorted(cc,key=operator.itemgetter(1),reversed=True)
sortedcc[0][0]"""

# 将labels的形状设置为(130,)
iris_y_train.shape=(130,)
iris_y_train

[5.1 3.8 1.5 0.3]
['Iris-setosa']


array(['Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica',
       'Iris-virginica', 'Iris-versicolor', 'Iris-virginica',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa

### 将训练集、测试集带入自定义KNN分类器进行分类

In [139]:
# 将训练集、测试集带入自定义KNN分类器进行分类
test_index=12
predict=KNNClassify(iris_x_test[test_index],iris_x_train,iris_y_train,20,"distance")
print (predict)
print ("新输入的实际类别是：", iris_y_test[test_index])
print ("\n")

if predict==iris_y_test[test_index]:
    print ("预测准确!")
else:
    print ("预测错误！")

新输入到训练集的最近20个点的距离加权计数为： 
 {'Iris-versicolor': 45.596003202769246}
新输入的类别是: Iris-versicolor
Iris-versicolor
新输入的实际类别是： ['Iris-versicolor']


预测准确!


#### 另一组简单的测试数据分类

In [140]:
# 另一组简单的测试数据分类
group = np.array([[1.0, 0.9], [1.0, 1.0], [0.1, 0.2], [0.0, 0.1]])
labels = np.array(['A', 'A', 'B', 'B'])
testX = np.array([1.2, 1.0])

KNNClassify(testX,group,labels,4,"distance")

新输入到训练集的最近4个点的距离加权计数为： 
 {'A': 9.472135954999581, 'B': 1.4018812887604746}
新输入的类别是: A


'A'

In [141]:
labels = np.array(['A', 'A', 'B', 'B'])
labels.shape

(4,)

In [142]:
iris_y_train.shape

(130,)